In [1]:
%config InteractiveShell.ast_node_interactivity="last_expr_or_assign"

In [2]:
import collections
import itertools
import string

Load the problem

In [3]:
def parse_segments(segments):
    return tuple(frozenset(s) for s in segments.split())

In [4]:
with open("input.txt") as f:
    puzzle = [((w := parse_segments(l))[:10], w[11:]) for l in f.readlines()]

Build a mapping of segments to digits

In [5]:
SEGMENTS_TO_DIGIT = {
    frozenset(k): v
    for k, v in {
        "abcefg": 0,
        "cf": 1,
        "acdeg": 2,
        "acdfg": 3,
        "bcdf": 4,
        "abdfg": 5,
        "abdefg": 6,
        "acf": 7,
        "abcdefg": 8,
        "abcdfg": 9,
    }.items()
}
DIGIT_TO_SEGMENTS = {v: k for k, v in SEGMENTS_TO_DIGIT.items()}

{0: frozenset({'a', 'b', 'c', 'e', 'f', 'g'}),
 1: frozenset({'c', 'f'}),
 2: frozenset({'a', 'c', 'd', 'e', 'g'}),
 3: frozenset({'a', 'c', 'd', 'f', 'g'}),
 4: frozenset({'b', 'c', 'd', 'f'}),
 5: frozenset({'a', 'b', 'd', 'f', 'g'}),
 6: frozenset({'a', 'b', 'd', 'e', 'f', 'g'}),
 7: frozenset({'a', 'c', 'f'}),
 8: frozenset({'a', 'b', 'c', 'd', 'e', 'f', 'g'}),
 9: frozenset({'a', 'b', 'c', 'd', 'f', 'g'})}

With a random pattern, only unordered properties are invariant. One such property is the segment frequencies:

In [6]:
CANONICAL_SEGMENT_FREQUENCIES = collections.Counter(
    itertools.chain.from_iterable(DIGIT_TO_SEGMENTS.values())
)

Counter({'g': 7, 'c': 8, 'e': 4, 'a': 8, 'f': 9, 'b': 6, 'd': 7})

Interestingly, when we treat each frequency as a score and take the sum for each digit, we observe a uniquely valued set:

In [7]:
DIGIT_TO_SCORE = {
    sum([CANONICAL_SEGMENT_FREQUENCIES[s] for s in segments]): i
    for i, segments in DIGIT_TO_SEGMENTS.items()
}

{42: 0, 17: 1, 34: 2, 39: 3, 30: 4, 37: 5, 41: 6, 25: 7, 49: 8, 45: 9}

This can be used to determine the true digits by building a frequency table for each set of patterns:

Compute the results!

In [8]:
def determine_digits(patterns, outputs):
    segment_frequencies = collections.Counter(itertools.chain.from_iterable(patterns))

    output_scores = [
        sum([segment_frequencies[s] for s in segments]) for segments in outputs
    ]

    return tuple([DIGIT_TO_SCORE[x] for x in output_scores])

In [9]:
digits = [determine_digits(p, o) for p, o in puzzle]

[(3, 1, 5, 0),
 (5, 3, 6, 0),
 (7, 0, 6, 9),
 (0, 3, 8, 5),
 (2, 3, 0, 3),
 (3, 2, 7, 3),
 (7, 1, 1, 8),
 (8, 6, 2, 9),
 (3, 0, 1, 9),
 (8, 3, 4, 4),
 (5, 2, 8, 2),
 (9, 8, 5, 9),
 (6, 2, 0, 4),
 (7, 8, 1, 4),
 (7, 4, 3, 3),
 (3, 2, 3, 7),
 (9, 6, 2, 0),
 (1, 5, 0, 4),
 (3, 9, 3, 2),
 (8, 4, 9, 3),
 (4, 8, 6, 2),
 (9, 4, 5, 4),
 (7, 9, 9, 7),
 (6, 2, 2, 3),
 (2, 1, 9, 3),
 (1, 5, 5, 9),
 (2, 8, 5, 8),
 (7, 3, 6, 3),
 (6, 1, 4, 1),
 (4, 1, 2, 9),
 (0, 9, 5, 1),
 (6, 6, 0, 5),
 (2, 2, 8, 6),
 (1, 4, 4, 9),
 (9, 9, 9, 6),
 (3, 0, 4, 5),
 (9, 2, 2, 5),
 (3, 9, 5, 1),
 (1, 8, 5, 7),
 (6, 7, 7, 9),
 (4, 2, 2, 0),
 (0, 6, 2, 4),
 (9, 1, 7, 8),
 (5, 1, 9, 9),
 (1, 0, 2, 6),
 (4, 2, 9, 3),
 (3, 5, 8, 7),
 (5, 8, 8, 4),
 (3, 4, 5, 5),
 (7, 0, 7, 5),
 (8, 6, 0, 9),
 (0, 2, 8, 7),
 (3, 0, 0, 3),
 (0, 7, 8, 9),
 (9, 8, 2, 3),
 (4, 9, 7, 2),
 (3, 9, 5, 3),
 (5, 5, 5, 3),
 (6, 2, 0, 8),
 (8, 3, 2, 1),
 (2, 1, 7, 3),
 (9, 4, 4, 8),
 (5, 2, 8, 4),
 (5, 0, 4, 4),
 (6, 9, 9, 9),
 (1, 5, 4, 6),
 (2, 6, 0,

Let's compute the aggregate frequencies

In [10]:
digit_frequencies = collections.Counter(itertools.chain.from_iterable(digits))

Counter({3: 84, 1: 84, 5: 74, 0: 86, 6: 74, 7: 75, 9: 77, 8: 74, 2: 95, 4: 77})

We need to convert our digits to integers (in base 10) before we can compute the total output value

In [11]:
def digits_to_int(digits):
    return sum((x * 10 ** i for i, x in enumerate(reversed(digits))))

In [12]:
output_total = sum([digits_to_int(d) for d in digits])

915941